In [1]:
import itertools
import string
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import tqdm
import pdb

import pandas as pd
from nltk.corpus import words


In [2]:
def fact(x):
    return np.linspace(1,x,x).prod()
def nck(n,k):
    return fact(n)/(fact(k)*fact(n-k))
nck(26,5),fact(4)

(65780.0, 24.0)

In [3]:
def new_condition(word,letter_status):
    good_letters = []
    for i,letter in enumerate(list(word)):
        if letter_status[i]<3:
            good_letters.append(letter)
    def inner_test(w):
        new_conditions = []
        for i,letter in enumerate(list(word)):
            if letter_status[i]==0:
                new_conditions.append(
                    letter in w 
                )                
            elif letter_status[i]==1:
                new_conditions.append(
                    letter in w[i] 
                )                
            elif letter_status[i]==2:
                new_conditions.append(
                    (letter in w) and (letter not in w[i])
                )
            else:
                new_conditions.append(
                    letter not in w
                )
        return new_conditions
    return inner_test,good_letters

class wordl():
    def __init__(self,show_progress=True):
        self.wordlist = []
        self.letter_list = []
        self.letter_status = []
        self.conditions = []
        self.show_progress = show_progress
        all_words = set(
            words.words()
        )
        self.nltk_words =  set(
            [
                wo 
                for wo in all_words
                if (len(wo)==5) and (wo.lower()==wo)
            ]
        )
    
    def add_word(self,word,letter_status,debug_word=''):
        self.wordlist.append(word)
        self.letter_status.append(letter_status)
        condition_list,letter_list = new_condition(word,letter_status)
        self.conditions.append(condition_list)
        self.letter_list.extend(letter_list)
        self.letter_list = list(set(self.letter_list))
    
    def conds(self,w):
        cond_results = []
        for i in range(len(self.conditions)):
            cond_results.extend(self.conditions[i](w))
        return all(cond_results)
    
#     def try_it(self,letter_list=None):
#         words = []
#         curr_lets = self.letter_list if letter_list is None else letter_list
#         # get all letters that have not yet been identified as letters in the target word
#         all_letters = set(string.ascii_lowercase).difference(set(curr_lets))
        
#         # create an enumeration of all permutations of these remaining letters by grouping them
#         #  into possible words where the size of each word is 5-len(curr_lets)
#         num_letters_left_to_solve = 5-len(curr_lets)
#         iterable = enumerate(itertools.permutations(all_letters, num_letters_left_to_solve))
#         # see if we want to show progress
#         if self.show_progress:
#             iterable = tqdm.tqdm(iterable)
                             
#         for i,v in iterable:
#             # v is a list of letters.  Make it a word
#             v2 = ''.join(v)
#             # find all ordering) of the letters in the word curr_lets+list(v)
#             word_to_permute = curr_lets+list(v)
#             for k in itertools.permutations(word_to_permute,5):
#                 # k is a list. make it a word
#                 k2 = ''.join(k)
#                 # find the word in nltk
#                 if k2 in self.nltk_words:
#                     # get the word part of the first value returned by nltk
#                     c1 = self.conds(k2)
#                     c2 = k2 not in words
#                     c3 = all([l in k2 for l in curr_lets])
#                     if all([c1,c2,c3]):
#                         words.append(k2)
#         return words

    def try_it(self,letter_list=None):
        words = []
        curr_lets = self.letter_list if letter_list is None else letter_list
        poss_words = [
            w for w in self.nltk_words
            if len(set(curr_lets).intersection(w))==len(curr_lets)
        ]
        for w in poss_words:
            if self.conds(w):
                words.append(w)
        return words
        
 

In [24]:
# uncomment to re-create df_wordl_words.csv
# wdl = wordl()
# pd.DataFrame({'word':list(wdl.nltk_words)}).to_csv('df_wordl_words.csv',index=False)


In [4]:
# DO_MAX_WORDS = True
# if DO_MAX_WORDS:
#     max_words = []
#     max_num_words = 0
#     for subset in tqdm.tqdm(itertools.combinations(string.ascii_lowercase,5)):
#         wd = wordl(show_progress=False)
#         word = ''.join(subset)
#         wd.add_word(word,[0,0,0,0,0])
#         words = wd.try_it()
#         if len(words)>max_num_words:
#             max_words = [word]
#             max_num_words = len(words)
#         if len(words)==max_num_words:
#             max_words.append(word)
#     wd = wordl(show_progress=False)
#     word = max_words[0]
#     wd.add_word(word,[0,0,0,0,0])
#     print(wd.try_it())
            

In [5]:
%%time
# infer
wdl = wordl()
wdl.add_word('slate',[3,3,3,3,2])
wdl.add_word('minor',[3,2,2,3,1])
# wdl.add_word('stare',[0,0,0,0,0])

wdl.try_it()

CPU times: user 126 ms, sys: 14.3 ms, total: 140 ms
Wall time: 143 ms


['infer', 'inker']

In [6]:
%%time
# joust
wdl = wordl()
wdl.add_word('slate',[2,3,3,2,3])
# t
wdl.add_word('minor',[3,3,3,2,3])
# ost
# wdl.add_word('stare',[0,0,0,0,0])

wdl.try_it()

CPU times: user 134 ms, sys: 5.95 ms, total: 140 ms
Wall time: 140 ms


['boost',
 'tsubo',
 'tossy',
 'tousy',
 'joust',
 'ghost',
 'toshy',
 'toosh',
 'coost']

In [7]:
%%time
# 
wdl = wordl()
wdl.add_word('slate',[2,3,3,2,3])
wdl.add_word('minor',[3,3,3,2,3])
# wdl.add_word('stare',[0,0,0,0,0])

wdl.try_it()

CPU times: user 134 ms, sys: 6.14 ms, total: 140 ms
Wall time: 145 ms


['boost',
 'tsubo',
 'tossy',
 'tousy',
 'joust',
 'ghost',
 'toshy',
 'toosh',
 'coost']

In [8]:
# braid
wdl = wordl()
wdl.add_word('slate',[3,3,1,3,3])
wdl.add_word('minor',[3,2,3,3,2])
# wdl.add_word('stare',[0,0,0,0,0])

wdl.try_it()

['vraic',
 'braid',
 'fraid',
 'graip',
 'ajari',
 'acari',
 'fraik',
 'izard',
 'urari']

In [9]:
wdl = wordl()
wdl.add_word('slate',[3,3,3,3,3])
wdl.add_word('minor',[3,3,2,2,3])
wdl.add_word('goony',[3,2,1,2,3])
wdl.try_it()

['knock', 'unown', 'known']

In [10]:
w = widgets.IntSlider()
display(w)
t = widgets.Text(value=str(w.value), disabled=True)
wt_link = widgets.jslink((w, 'value'), (t, 'value'))
display(t)

IntSlider(value=0)

Text(value='0', disabled=True)

In [11]:
%%html
<style>
.mytext {
    font-weight: bold;
    color: blue;
    font-size:100px;
    text-align:center;
}
</style>


In [17]:
table_style = {'description_width': 'auto'}
table_layout = {'width':'auto', 'min_width':'15px',  'min_height':'28px'}

num_tboxes = 5
tboxes = [
    widgets.Text(
        layout=table_layout,
        style=table_style
    )
    for i in range(num_tboxes)
]  

dropdowns = [
    widgets.Dropdown(
        options=[str(i) for i in [0,1,2,3]],
        value='3',
        layout=table_layout,
        style=table_style
    )
    for _ in range(num_tboxes)
]

wd_box = wordl()

def on_add_word_button_clicked(change):
    print('-----------')
    word = ''.join(
        [
            tbox.value
            for tbox in tboxes
        ]
    )
    nums = [
        int(dropdown.value)
        for dropdown in dropdowns
    ]
    print(word,nums)
    wd_box.add_word(word,nums)

button_add_word = widgets.Button(
    description='add_word',
    layout=widgets.Layout(width='auto', height='auto')
)
button_add_word.on_click(on_add_word_button_clicked)


def on_try_it_button_clicked(change):
    words = wd_box.try_it()
    print(words)

button_tryit = widgets.Button(
    description='try_it',
    layout=widgets.Layout(width='auto', height='auto')
)
button_tryit.on_click(on_try_it_button_clicked)

perc=35
gbox1 = widgets.GridBox(
#     children=tboxes+dropdowns+[widgets.HBox([button_add_word,button_tryit])],
    children=tboxes+dropdowns,
    layout=widgets.Layout(
        display='grid',
        width=f'{perc}%',
        height='auto',
        grid_template_rows='1fr 1fr',
        grid_template_columns=' '.join(['auto' for _ in range(len(tboxes))])
    )
)

for tbox in tboxes:
    tbox.add_class('mytext')

gbox2 = widgets.HBox(
    children=[button_add_word,button_tryit],
    layout=widgets.Layout(
        display='grid',
        width=f'{perc}%',
        height='auto',
        grid_template_rows='1fr',
        grid_template_columns=' '.join(['auto' for _ in range(2)])
    )
)


gbox3 = widgets.VBox([gbox1,gbox2],width=f'{perc}%')
display(gbox3)

-----------
slate [3, 3, 3, 3, 3]
-----------
minor [3, 3, 2, 2, 3]
-----------
goony [3, 2, 1, 2, 3]
['knock', 'unown', 'known']


In [14]:
%%time
ss = [
    w
    for w in wdl.nltk_words
    if len(set(w))<5
]
len(ss)

CPU times: user 4.03 ms, sys: 49 µs, total: 4.08 ms
Wall time: 4.36 ms


3003